In [2]:
import dspy

/home/enakilci/miniforge3/envs/llm-assistant/lib/python3.13/site-packages/pydantic/_internal/_config.py:345: UserWarning: Valid config keys have changed in V2:
* 'fields' has been removed
  warnings.warn(message, UserWarning)
/home/enakilci/miniforge3/envs/llm-assistant/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
backend_llm = dspy.OllamaLocal(
    base_url='http://10.7.0.13:11434',  # This is where the Ollama server is located.
    model='fixed_qwen1_5-14b-chat-q5_k_m:latest',  # The model we're going to use.
    num_ctx=16000,  # How much context the model can consider.
    temperature=0.6,  # This adjusts how creative the model's responses are.
    timeout_s=180,  # How long we wait for a response before giving up.
    max_tokens=2048,  # The maximum length of the model's responses.
    stop=['---','Explanation:','<|im_start|>','<|im_end|>'] # Characters or phrases that signal the model to stop generating text.
)

In [3]:
#connect to LM from your DSPy code
options = {"num_ctx": 1024}
lm = dspy.LM('ollama_chat/deepseek-r1:7b', api_base='http://localhost:11434', num_ctx= 8192*2)
dspy.configure(lm=lm)

In [4]:
#Calling LM directly
lm("Say this is a test!", temperature=0.7)  # => ['This is a test!']
lm(messages=[{"role": "user", "content": "Say this is a test in Turkish three times."}])  # => ['This is a test!']

['<think>\nOkay, so I need to figure out how to respond to the user\'s request where they asked me to say "This is a test in Turkish" three times. Hmm, let\'s break this down.\n\nFirst, I should understand what exactly the user is asking for. They want me to repeat that specific sentence in Turkish three times. So my response needs to be straightforward and repetitive as per their instruction.\n\nWait, but why would someone ask that? Maybe they\'re testing if I can do basic language translation or if I\'m paying attention. Alternatively, it could be part of a larger task or assessment. Regardless, the immediate request is clear: repeat the sentence three times in Turkish.\n\nI should make sure each repetition is correct and uses proper Turkish grammar. Let me think about how "This is a test in Turkish" translates into Turkish. The phrase would be "Bu bir Türkçe test." So I need to say that three times accurately.\n\nAlso, considering the user might want it in a specific format or with 

# Math

In [5]:
math = dspy.ChainOfThought("question -> answer:float")
output = math(question="Two dice are tossed. What is the probability that the sum equals two?Explain to me in detail")
print(output)

Prediction(
    reasoning='Each die has six faces (1 to 6). The total number of possible outcomes when tossing both dice is 6 * 6 = 36. Only one outcome (both dice showing 1) results in a sum of two. Therefore, the probability is 1/36 ≈ 0.0278.',
    answer=0.0278
)


In [6]:
# Converting the fraction to a float programmatically:
import fractions

math = dspy.ChainOfThought("question -> answer")
output = math(question="Two dice are tossed. What is the probability that the sum equals two?")

# Convert fraction to float
try:
    probability = float(fractions.Fraction(output.answer))
    print(probability)  # Expected: 0.027777...
except ValueError:
    print("Could not convert output to float:", output.answer)


Could not convert output to float: The probability that the sum equals two when two dice are tossed is \boxed{\dfrac{1}{36}}.


In [7]:
math = dspy.ChainOfThought("question -> reasoning:str, answer:float")

output = math(question="Two dice are tossed. What is the probability that the sum equals two? ")

print(output)  # Should return a structured output


Prediction(
    reasoning="To find the probability that the sum of two dice equals two, we first determine the total number of possible outcomes when tossing two dice, which is \\(6 \\times 6 = 36\\). The only favorable outcome for a sum of two is both dice showing 1. Thus, there's 1 favorable outcome out of 36 possible outcomes.",
    answer=0.0278
)


In [8]:
output.answer

0.0278

# RAG

In [9]:
def search_wikipedia(query: str) -> list[str]:
    results = dspy.ColBERTv2(url='https://en.wikipedia.org/wiki/Not_Like_Us?action=edit')(query, k=3)
    return [x['text'] for x in results]

rag = dspy.ChainOfThought('context, question -> response')

question = "For which artist Kendrick Lamar wrote the diss track 'Not Like Us'?"
rag(context=search_wikipedia(question), question=question)

JSONDecodeError: Expecting value: line 1 column 1 (char 0)

In [13]:
import wikipediaapi

def search_wikipedia(query: str) -> str:
    wiki_wiki = wikipediaapi.Wikipedia(
        language='en',
        user_agent="LLM-Assistant/1.0 (ecagatay93@email.com)"
    )
    page = wiki_wiki.page(query)

    if not page.exists():
        return "No relevant Wikipedia page found."

    return page.text[:10000]  # Limit content for processing

# Example usage
# question = "For which artist did Kendrick Lamar write the diss track 'Not Like Us'?"
context = search_wikipedia("2024 United States presidential election")

print(len(context) ) # Print a preview of the content


10000


In [14]:
print(context)

Presidential elections were held in the United States on November 5, 2024. The Republican Party's ticket—Donald Trump, who was the 45th president of the United States from 2017 to 2021, and JD Vance, the junior U.S. senator from Ohio—defeated the Democratic Party's ticket—Kamala Harris, the incumbent vice president, and Tim Walz, the 41st governor of Minnesota.
The incumbent president, Joe Biden of the Democratic Party, initially ran for re-election as the party's presumptive nominee, facing little opposition and easily defeating Representative Dean Phillips during the Democratic primaries; however, what was broadly considered a poor debate performance in June 2024 intensified concerns about his age and health, and led to calls within his party for him to leave the race. After initially declining to do so, Biden withdrew on July 21, becoming the first eligible incumbent president to withdraw since Lyndon B. Johnson in 1968. Biden endorsed Harris, who was voted the party's nominee by th

In [15]:
rag = dspy.ChainOfThought('context, question -> response')

question = "What is the name of running mate of Donald Trump in 2024?"
rag(context=context, question=question)

Prediction(
    reasoning="The context mentions that Donald Trump ran for re-election in 2024 and was nominated along with his running mate, JD Vance. The article does not mention any other candidates as Trump's running mate.",
    response='JD Vance'
)

In [16]:
rag = dspy.ChainOfThought('context, question -> response')

question = "Did trump claim that the 2020 elections were rigged?"
rag(context=context, question=question)

Prediction(
    reasoning='The context provided mentions that Donald Trump falsely claimed there was voter fraud in the 2020 presidential election, which he denied after losing. This includes his baseless predictions about voter fraud in 2024 as well as his claims of election interference.',
    response='Yes, Trump claimed that the 2020 election was rigged and made false claims about voter fraud to deny the results.'
)

# Classification

In [26]:
from typing import Literal

class Classify(dspy.Signature):
    """Classify sentiment of a given sentence."""

    sentence: str = dspy.InputField()
    sentiment: Literal['positive', 'negative', 'neutral'] = dspy.OutputField()
    confidence: float = dspy.OutputField()

classify = dspy.Predict(Classify)
classify(sentence="This book was super fun to read")

Prediction(
    sentiment='positive',
    confidence=0.8
)

# Information Extraction

In [23]:
class ExtractInfo(dspy.Signature):
    """Extract structured information from text."""

    text: str = dspy.InputField()
    title: str = dspy.OutputField()
    headings: str = dspy.OutputField(desc="a JSON string representing a list of headings")
    entities: str = dspy.OutputField(desc="a JSON string representing a list of entities and their metadata")


In [ ]:
import json

module = dspy.Predict(ExtractInfo)
text = "Apple Inc. announced its latest iPhone 14 today." \
    "The CEO, Tim Cook, highlighted its new features in a press release."
response = module(text=text)

print("Raw response:", repr(response))
print(response.title)
print(response.headings)
print(response.entities)


In [137]:
class ExtractInfo(dspy.Signature):
    """Extract structured information from text."""

    text: str = dspy.InputField()
    title: str = dspy.OutputField()
    headings: list[str] = dspy.OutputField()
    entities: list[dict[str, str]] = dspy.OutputField(desc="a list of entities and their metadata")

In [138]:
import json

module = dspy.Predict(ExtractInfo)
text = "securite securite all stations this is United States Coast Guard Sector Long Island Sound break the coast guard recieved a report of a vessel that is submerged in the vicinity of Sag Harbor entrance in broad position four one tag zero one decimal eight two north zero seven two tag one eight decimal zero zero west all mariners are requested caution when transiting the area break this is United States Coast Guard Sector Long Island Sound out"
response = module(text=text)

print("Raw response:", repr(response))
print(response.title)
print(response.headings)
print(response.entities)

Raw response: Prediction(
    title='Submerged Vessel Incident Near Sag Harbor Entrance',
    headings=['Vessel Identification', 'Position', 'Mariner Caution'],
    entities=[{'name': 'Vessel Tag', 'value': 'tag zero one decimal eight two north zero seven two'}, {'name': 'Position', 'value': 'four one tag zero one decimal eight two north zero seven two west'}, {'name': 'Direction', 'value': 'zero zero west'}, {'name': 'Warning', 'value': 'break the coast guard recieved a report of a vessel that is submerged in the vicinity of Sag Harbor entrance'}]
)
Submerged Vessel Incident Near Sag Harbor Entrance
['Vessel Identification', 'Position', 'Mariner Caution']
[{'name': 'Vessel Tag', 'value': 'tag zero one decimal eight two north zero seven two'}, {'name': 'Position', 'value': 'four one tag zero one decimal eight two north zero seven two west'}, {'name': 'Direction', 'value': 'zero zero west'}, {'name': 'Warning', 'value': 'break the coast guard recieved a report of a vessel that is submer

In [1]:
class ExtractInfo(dspy.Signature):
    """Extract structured information from text."""

    text: str = dspy.InputField()
    title: str = dspy.OutputField()
    headings: str = dspy.OutputField(desc="a JSON string representing a list of headings")
    entities: str = dspy.OutputField(desc="a JSON string representing a list of entities and their metadata")

NameError: name 'dspy' is not defined

In [141]:
import json

module = dspy.Predict(ExtractInfo)
text = "Kotka VTS, Onki, hyvää iltapäivää. Vessel Strandvej, this is Tavasland."
response = module(text=text)

print("Raw response:", repr(response))
print(response.title)
print(response.headings)
print(response.entities)

Raw response: Prediction(
    title='Kotka VTS',
    headings='[\n  "Kotka VTS",\n  "Vessel Strandvej"\n]',
    entities='[\n  {\n    "name": "Kotka VTS",\n    "type": "ship_name",\n    "description": "A vessel named Kotka VTS."\n  },\n  {\n    "name": "Onki",\n    "type": "person_name",\n    "description": "A person\'s name, Onki."\n  },\n  {\n    "name": "Tavasland",\n    "type": "location_or_area",\n    "description": "A location or area named Tavasland."\n  }\n]'
)
Kotka VTS
[
  "Kotka VTS",
  "Vessel Strandvej"
]
[
  {
    "name": "Kotka VTS",
    "type": "ship_name",
    "description": "A vessel named Kotka VTS."
  },
  {
    "name": "Onki",
    "type": "person_name",
    "description": "A person's name, Onki."
  },
  {
    "name": "Tavasland",
    "type": "location_or_area",
    "description": "A location or area named Tavasland."
  }
]


# Agents

In [143]:
def evaluate_math(expression: str):
    return dspy.PythonInterpreter({}).execute(expression)

def search_wikipedia(query: str):
    results = dspy.ColBERTv2(url='http://20.102.90.50:2017/wiki17_abstracts')(query, k=3)
    return [x['text'] for x in results]

react = dspy.ReAct("question -> answer: float", tools=[evaluate_math, search_wikipedia])

pred = react(question="What is 9362158 divided by the year of birth of David Gregory of Kinnairdy castle?")
print(pred.answer)

KeyboardInterrupt: 

# Multi-Stage Pipelines